In [ ]:
import pandas as pd
import datetime
import os, shutil, pathlib, fnmatch
import mysql.connector as msql
from mysql.connector import Error


def read_and_process(flight_data_file):
    '''
    In this function only columns 'origin', 'destination', 'day' are selected from the dataset.
    Then the dataset is cleaned using function clean_flight_data and finally data is inserted using
    database using function insert_flight_data.
    '''
    flight_data = pd.read_csv(flight_data_file, usecols = ['origin', 'destination', 'day'])
    flight_data = clean_flight_data(flight_data)
    insert_flight_data(flight_data)
    return

def clean_flight_data(input_data):
    '''
    In this function from the origin column null values are dropped and day column converted to datetime format.
    New columns year, month and dd created. Finally, the null values that are present in destination column is set
    to notnull, so that data can be inserted to MYSQL database.    
    '''
    flight_data = input_data.dropna()
    flight_data.day = pd.to_datetime(flight_data.day)
    flight_data['year'] = flight_data['day'].dt.year
    flight_data['month'] = flight_data['day'].dt.month
    flight_data['dd'] = flight_data['day'].dt.day
    return flight_data

def process_flight_data_folder(filepath, pattern: str = '*'):
    '''
    In this function using for loop iterate through each file then use function read_and_process and read files.
    Then using function move_files_to_processed and move the processed files to separate folder 'processed'.
    '''
    for filename in fnmatch.filter(os.listdir(filepath), pattern):
        read_and_process(filepath + "\\" + filename)
        print(filename + " processed")
    move_files_to_processed(filepath)
    return    

def move_dir(src: str, dst: str, pattern: str = '*'):
    '''
    In this function files are moved to the designated folder.
    '''
    if not os.path.isdir(dst):
        pathlib.Path(dst).mkdir(parents=True, exist_ok=True)
    for f in fnmatch.filter(os.listdir(src), pattern):
        shutil.move(os.path.join(src, f), os.path.join(dst, f))
        
def move_files_to_processed(filepath):
    '''
    In this function select the filepath and folder and call out function move_dir to move the files.    
    '''
    move_to = filepath + "processed\\"
    move_dir(filepath, move_to, '*.csv' )
    print("Moved files successfully to " + move_to)
    return


def read_country_data(country_file_path):
    '''
    In this function, airport code data is read into country data variable.
    '''
    country_data = pd.read_excel(country_file_path)
    return country_data

def connect_db():
    '''
    In this function, using mysql.connect function, we are connecting to MYSQL Sever.
    '''
    try:
        conn = msql.connect(host='localhost', user='abhi', passwd="Abhi2022$")

    except Error as e:
        
        print("Error while connecting to MySQL", e)
    return conn

     

def create_database_tables():
    '''
    In this function, flight_database and tables flights and countries are created in MySQL Workbench.
    '''
    
    try:
    
        conn = connect_db()

    #To connect to the MySQL Workbench and create a flight_data database
        if conn.is_connected():
            cursor = conn.cursor()
            cursor.execute("DROP DATABASE IF EXISTS flight_database_01")
            cursor.execute("CREATE DATABASE flight_database_01")
            print("flight_database_01 database is created")
            cursor.execute("use flight_database_01")
            print("You're connected to database: ")
            
    #To create "countries" table in the flight_databse created above.
            cursor.execute('DROP TABLE IF EXISTS countries;')
            print('Creating countries table....')
            cursor.execute('''CREATE TABLE IF NOT EXISTS `flight_database_01`.`countries` (
              `airport_code` VARCHAR(10) NOT NULL,
              `country` VARCHAR(45) NOT NULL,
              `continent` VARCHAR(45) NOT NULL,
              PRIMARY KEY (`airport_code`)
              )''')
            print("\"countries\" table is created....")

    #To create "flights" table in the flight_databse created above. 
            cursor.execute('DROP TABLE IF EXISTS flights;')
            print('Creating flights table....')
            cursor.execute('''CREATE TABLE IF NOT EXISTS `flight_database_01`.`flights` (
              `id` INT NOT NULL AUTO_INCREMENT,
              `origin` VARCHAR(10) NOT NULL,
              `destination` VARCHAR(10) NULL,
              `date` DATETIME NOT NULL,
              `year` INT NOT NULL,
              `month` INT NOT NULL,
              `day` INT NOT NULL,
              PRIMARY KEY (`id`),
              INDEX `origin_idx` (`origin` ASC) VISIBLE,
              INDEX `destination_idx` (`destination` ASC) VISIBLE
              )''')
            print("\"flights\" table is created....")

 
    #The connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()

    except Error as e:
        print("Error while creating table in MySQL", e)
    return

def insert_country_data(country_data):
    '''
    In this function, data from the dataset is inserted into the flights table in the flight_database.
    '''
    
    try:
        
        conn = connect_db()

#To connect to the MySQL Workbench and create a flight_data database
        if conn.is_connected():
            cursor = conn.cursor()
            #Here we are iterating through the rows and inserting the data into database. 
        for i, row in country_data.iterrows():
            sql = "INSERT INTO flight_database_01.countries (airport_code, country, continent) VALUES (%s,%s,%s)"
            cursor.execute(sql, tuple(row))
        print("Data inserted into \"countries\" table")
        
#The connection is not autocommitted by default, so we must commit to save our changes
        conn.commit()
    
    except Error as e:
        print("Error while inserting country data in MySQL", e)
        
    return
    

def insert_flight_data(flight_data):
    '''
    In this function, data from the dataset is inserted into the flights table in the flight_database.
    '''
    
    try:
        
        conn = connect_db()

#To connect to the MySQL Workbench and create a flight_data database
        if conn.is_connected():
            cursor = conn.cursor()
            
#Here we are iterating through the rows and inserting the data into database. 
        for i, row in flight_data.iterrows():
            sql = "INSERT INTO flight_database_01.flights (origin, destination, date, year, month, day) VALUES (%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
        print("Data inserted into \"flights\" table")

        
#The connection is not autocommitted by default, so we must commit to save our changes
        conn.commit()
    
    except Error as e:
        print("Error while inserting flight data in MySQL", e)
    return


In [ ]:
#Run this code for first time only, then only run the function "process_flight_data_folder(flights_filepath,'*.csv')" for every
#new file placed into the designated filepath.

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
startTime = datetime.now()

filepath = "E:\\IGP DATA MIGRATION\\"
flights_filepath = filepath + 'flight_data/'
country_file_path = filepath + 'airport_code.xlsx'

create_database_tables()
country_data = read_country_data(country_file_path)
insert_country_data(country_data)
process_flight_data_folder(flights_filepath,'*.csv')

print ('Time taken to process this code :', datetime.now() - startTime)

In [ ]:
#Run this code for every new file placed into the designated filepath.

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
startTime = datetime.now()

filepath = "E:\\IGP DATA MIGRATION\\"
flights_filepath = filepath + 'flight_data/'

process_flight_data_folder(flights_filepath,'*.csv')

print ('Time taken to process this code :', datetime.now() - startTime)